In [ ]:
# 1. Install the required packages
!pip install flask-ngrok flask-cors pyngrok
%pip install -U segment-geospatial

In [ ]:
# Import necessary libraries for ngrok
from pyngrok import ngrok, conf
import os

# Set your ngrok auth token here.
# Replace 'YOUR_AUTH_TOKEN' with your actual token from ngrok.com
ngrok.set_auth_token('YOUR_AUTH_TOKEN')

# Configure ngrok to use the European region for the tunnel
conf.get_default().region = 'eu'

In [ ]:
# Import necessary libraries
from flask import Flask, request, jsonify  # For creating the web application and handling requests/responses
from flask_cors import CORS  # To enable Cross-Origin Resource Sharing
import os  # For operating system interactions (e.g., deleting files)
from samgeo import SamGeo2, regularize  # For using the Segment Anything Model (SAM) and refining building shapes
import tempfile  # For creating temporary files during processing
import json  # For handling JSON data
import numpy as np  # For array operations
import rasterio  # For handling geospatial raster data

# Create a Flask web application instance
app = Flask(__name__)
# Enable Cross-Origin Resource Sharing for the app
CORS(app)

# Define the API route for building detection
@app.route('/detect', methods=['POST'])  # This route accepts POST requests
def detect():
    try:
        # Check if the image file is in the request
        if 'image' not in request.files:
            return jsonify({'error': 'No image file provided'}), 400

        # Get the points data from form data and parse it as JSON
        if 'points' not in request.form:
            return jsonify({'error': 'No points data provided'}), 400

        points = json.loads(request.form['points'])
        print(f"Received points: {points}")  # Points are already in [lon, lat] format

        # Create a temporary file to store the uploaded image
        with tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_image:
            # Save the uploaded image to the temporary file
            request.files['image'].save(temp_image.name)
            # Get the path of the temporary image file
            image_path = temp_image.name
            print(f"Saved image to: {image_path}")

        # Initialize the SAM2 model with the specified model ID
        sam = SamGeo2(
            model_id="sam2-hiera-large",
            automatic=False,  # Set automatic prediction to False
        )

        # Set the image for the SAM model to process
        sam.set_image(image_path)
        print("SAM model initialized and image set")

        # Create temporary files for intermediate and final outputs
        with tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_mask, \
            tempfile.NamedTemporaryFile(suffix='.geojson', delete=False) as temp_vector, \
            tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_buildings, \
            tempfile.NamedTemporaryFile(suffix='.geojson', delete=False) as output_regularized:

            # Run building detection using SAM with the provided points (already in correct format)
            print("Starting prediction with points...")
            sam.predict_by_points(
                point_coords_batch=points,  # Points are already in [lon, lat] format
                point_crs="EPSG:4326",  # Specify coordinate reference system
                output=temp_mask.name,  # Save the output mask to a temporary file
                dtype="uint8",  # Set data type for the output mask
            )
            print(f"Prediction completed, mask saved to: {temp_mask.name}")

            # Group detected regions to form building outlines
            print("Starting region grouping...")
            array, gdf = sam.region_groups(
                temp_mask.name,  # Use the generated mask
                min_size=200,  # Set minimum size for regions
                out_vector=temp_vector.name,  # Save vector data to a temporary file
                out_image=temp_buildings.name  # Save building image to a temporary file
            )
            print(f"Region grouping completed, vector saved to: {temp_vector.name}")

            # Refine building shapes for better accuracy
            print("Starting shape regularization...")
            regularize(temp_vector.name, output_regularized.name)
            print(f"Regularization completed, saved to: {output_regularized.name}")

            # Read the final output (likely GeoJSON data)
            with open(output_regularized.name, 'rb') as f:
                result = f.read()
            print("Result read successfully")

        # Delete temporary files to clean up
        os.unlink(image_path)
        os.unlink(temp_mask.name)
        os.unlink(temp_vector.name)
        os.unlink(temp_buildings.name)
        os.unlink(output_regularized.name)

        # Return the result with a success status code (200) and JSON content type
        return result, 200, {'Content-Type': 'application/json'}

    except Exception as e:
        import traceback
        print(f"Error in detect: {str(e)}")
        print(traceback.format_exc())
        return jsonify({'error': str(e)}), 500  # Return an error message with a 500 status code


In [ ]:
# start ngrok
public_url = ngrok.connect(5000)
print(f'Public URL: {public_url}')

# start Flask
app.run()